In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

# Generates a binary PUF response
def generate_puf_response(size):
    return np.random.randint(0, 2, size)

# Function to binarize weights to +1/-1
def binarize_weights(weights):
    return np.where(weights >= 0, 1, -1)

# Function to apply row inversion
def apply_row_inversion(weights, puf_response):
    transformed_weights = np.copy(weights)
    for j in range(weights.shape[0]):
        if puf_response[j] == 1:
            transformed_weights[j, :] *= -1
    return transformed_weights

# Function to apply column inversion (including BN layer transformation)
def apply_column_inversion(weights, puf_response, bn_beta):
    transformed_weights = np.copy(weights)
    transformed_bn_beta = np.copy(bn_beta) if bn_beta is not None else None
    for k in range(weights.shape[1]):
        if puf_response[k] == 1:
            transformed_weights[:, k] *= -1
            if transformed_bn_beta is not None:
                transformed_bn_beta[k] = -bn_beta[k] + 2
    return transformed_weights, transformed_bn_beta

In [2]:

# Loading MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.reshape(-1, 28*28) / 255.0, x_test.reshape(-1, 28*28) / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

model = Sequential([
    tf.keras.Input(shape=(28*28,)),
    Dense(256, activation='tanh'),
    BatchNormalization(),
    Dense(128, activation='tanh'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=64, verbose=1)

dense_layers = [layer for layer in model.layers if isinstance(layer, Dense)]
original_weights = [layer.get_weights()[0] for layer in dense_layers]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.8912 - loss: 0.3612
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9694 - loss: 0.1044
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9790 - loss: 0.0679


In [3]:
# Extracting BN parameters
bn_layers = [layer for layer in model.layers if isinstance(layer, BatchNormalization)]
bn_params = [layer.get_weights() for layer in bn_layers]

# Generating PUF responses
row_puf_responses = [generate_puf_response(w.shape[0]) for w in original_weights]
column_puf_responses = [generate_puf_response(w.shape[1]) for w in original_weights]


In [4]:
# Applying row inversion transformation
row_inverted_weights = [apply_row_inversion(binarize_weights(w), row_puf_responses[i]) for i, w in enumerate(original_weights)]

# Applying column inversion transformation
column_inverted_weights = []
transformed_bn_params = []
bn_idx = 0

for i, w in enumerate(original_weights):
    transformed_weight, transformed_bn = apply_column_inversion(binarize_weights(w), column_puf_responses[i], bn_params[bn_idx][1] if bn_idx < len(bn_params) else None)
    column_inverted_weights.append(transformed_weight)

    if bn_idx < len(bn_params):
        modified_bn = bn_params[bn_idx].copy()
        modified_bn[1] = transformed_bn
        transformed_bn_params.append(modified_bn)
        bn_idx += 1

In [7]:
# Function to set model weights
def set_model_weights(model, weight_list, bn_param_list=None):
    dense_idx = 0
    bn_idx = 0

    for layer in model.layers:
        if isinstance(layer, Dense):
            layer.set_weights([weight_list[dense_idx], layer.get_weights()[1]])  # Update weights, keep bias same
            dense_idx += 1
        elif isinstance(layer, BatchNormalization) and bn_param_list is not None and bn_idx < len(bn_param_list):
            layer.set_weights(bn_param_list[bn_idx])  # Restore full BN state (beta, gamma, moving mean, moving variance)
            bn_idx += 1

# Recalibrating BN Layers after Reset
def recalibrate_bn(model, x_train):
    _ = model.predict(x_train[:1000])  # BN layers to recalculate running mean/variance

# Printing sample weights
print("\n Original Weights (First 5 Rows of First Layer):")
print(original_weights[0][:5, :5])  # Prints top-left 5x5 block

print("\n Row-Inverted Weights (First 5 Rows of First Layer):")
print(row_inverted_weights[0][:5, :5])  # Prints transformed top-left 5x5 block

print("\n Column-Inverted Weights (First 5 Rows of First Layer):")
print(column_inverted_weights[0][:5, :5])  # Prints transformed top-left 5x5 block


 Original Weights (First 5 Rows of First Layer):
[[-0.04834355 -0.0236179  -0.05918679 -0.0700252  -0.02417278]
 [-0.02399136 -0.0045144  -0.02999295 -0.00105073 -0.02742407]
 [ 0.00968273  0.06135601 -0.05243165 -0.0172565   0.06411999]
 [ 0.06873459  0.06485483  0.04170287 -0.03234712  0.07265869]
 [-0.03718681 -0.01764654  0.0632728  -0.03513824 -0.02239991]]

 Row-Inverted Weights (First 5 Rows of First Layer):
[[ 1  1  1  1  1]
 [-1 -1 -1 -1 -1]
 [-1 -1  1  1 -1]
 [-1 -1 -1  1 -1]
 [-1 -1  1 -1 -1]]

 Column-Inverted Weights (First 5 Rows of First Layer):
[[ 1 -1  1 -1  1]
 [ 1 -1  1 -1  1]
 [-1  1  1 -1 -1]
 [-1  1 -1 -1 -1]
 [ 1 -1 -1 -1  1]]


In [8]:
# Evaluating Row-Inverted Model
set_model_weights(model, row_inverted_weights)
loss, row_inv_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\nAccuracy after Row Inversion: {row_inv_acc * 100:.2f}%")

# Evaluating Column-Inverted Model
set_model_weights(model, column_inverted_weights, transformed_bn_params)
loss, col_inv_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy after Column Inversion: {col_inv_acc * 100:.2f}%")

# Original Model
set_model_weights(model, original_weights, bn_params)
recalibrate_bn(model, x_train)
loss, original_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Original Model Accuracy: {original_acc * 100:.2f}% ")



Accuracy after Row Inversion: 9.72%
Accuracy after Column Inversion: 7.19%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Original Model Accuracy: 97.44% 
